**Start by running the cell below to import necessary packages and setup the testing enviornment**

In [ ]:
using Test, Plots, Distributions, LinearAlgebra, CSV, Random
using DataFrames, DecisionTree, StatsPlots, StatsBase, FreqTables
gr();

# IL027 Computer Modelling for All

## Assignment 3 - deadline 31st Jan 2020

Topics covered:
* probability distributions
* stochastic model
* data analysis
* predictive models

**NOTE:** Unlike the lecture notebooks, where we ask you to make a copy before making changes, for the assignments please don't make a copy but instead *make changes directly within this notebook*. For the assignment submission to work correctly, the filenames must match the original ones, i.e. `A3.ipynb` for this one.

### Part a - Probability Distributions and Stochastic Models

**Question 1 (5%)**

Create a vector `x` of length `10` containing random numbers drawn from the normal distribution with mean $0$ and variance $1$. 


In [ ]:
## BEGIN ANSWER 
x = randn(10)
## END ANSWER

**Question 2. (10%)**

Create a function `rexp(N)` that returns a vector `x` of length `N` containing random numbers drawn from the [exponential distribution](https://en.wikipedia.org/wiki/Exponential_distribution) with parameter $\lambda = 1.234$.  (You may use either `randexp` or `Distributions.jl`.)

In [ ]:
## BEGIN SOLUTION 
function rexp(N)
    Dexp = Exponential(1.234)
    return rand(Dexp, N)
end 
## END SOLUTION 

In [ ]:
## BEGIN HIDDEN TESTS
N = 10_000_000
Dexp = Exponential(1.234)
x = rexp(N)
y = rand(Dexp, N)
for n = 1:5
    println(@test abs(moment(x, n) - moment(y, n)) / (1.0+moment(x,n)) < 0.03)
end
## END HIDDEN TESTS

**Question 3. (15%)**

[1] form a vector `y` of length `M` where each entry is of the form 
$$
    y_i = \frac{1}{N} \sum_{j = 1} x_{ij},
$$
where $x_{ij}$ are independent random numbers drawn from Exp(1.234$. (Note: you can reuse the answer from Q2, but you don't have to!)

[2] fit a normal distribution to the random numbers stored in the vector `y`.

[3] Compute the `mean` and `variance` of the normal distribution and store them in the variables `m_fit, v_fit`.

NOTE: cf. `using Distributions`, `?Normal` `?mean`, `?var`

In [ ]:
M = 1_000
N = 1_000
y = zeros(M)
## BEGIN SOLUTION 
for m = 1:M
    y[m] = mean(rexp(N))
end 
Dn = fit(Normal, y)
m_fit = mean(Dn)
v_fit = var(Dn)
## END SOLUTION 
;

In [ ]:
## BEGIN HIDDEN TESTS
m_ex, v_ex = 1.234, 0.001526
println(@test abs(m_ex - m_fit) < 0.1)
println(@test abs(v_ex - v_fit)/abs(v_ex) < 0.1)
## END HIDDEN TESTS

In [ ]:
# you can draw a histogram to test your fit, 
# it is interesting to experiment with M, N and nbins
# Which fundamental theorem of Probability theory can you see at work here?
histogram(y, nbins=30, normed=true, label="data")
plot!( x -> pdf(Dn, x), lw=3, label="fit" )

**Question 4. (20%)**

Suppose we roll $n$ traditional dice, what is the 
probability of obtaining a certain number $m$ as the
sum of their faces? This can really be treated as a problem 
of combinatorics, but we will solve it using a  simple 
Monte Carlo simulation.

1. Implement a function `diceroll(n)` that simulates rolling `n` traditional dice and returns the sum of their faces.

2. Implement a function `mc_dice(n, m)` that uses a Monte Carlo method to approximately determine the probability of throwing the sum `m` with `n` dice and returns it.

3. Adjust the number of random samples used by your `mc_dice()` function so that you can be sure (reasonably confident) that the accuracy of your result is at least 0.01.

In [ ]:
## BEGIN SOLUTION 
diceroll(n) = sum( rand(1:6, n) )

function mc_dice(n, m, Nsamples = 100_000)
    count = 0 
    for i = 1:Nsamples 
        if diceroll(n) == m 
            count += 1
        end 
    end 
    return count / Nsamples 
end 
## END SOLUTION

In [ ]:
## BEGIN HIDDEN TESTS
diceroll_(n) = sum( rand(1:6, n) )

function mc_dice_(n, m, Nsamples = 1_000_000)
    count = 0
    for i = 1:Nsamples
        if diceroll_(n) == m
            count += 1
        end
    end
    return count / Nsamples
end

for (n, m) in zip((2, 3, 3, 4, 4), (5, 8, 6, 10, 15))
    sol = mc_dice(n, m)
    ex = mc_dice_(n, m)
    println(@test abs(sol - ex) < 0.02)
end
## END HIDDEN TESTS

In [ ]:
# test code which you can modify if you wish

println("Test 1")
n = 2 
m = 5  # 1+4, 2+3, 3+2, 3+1 => 4/36 = 1/9 
for i = 1:3
    @show mc_dice(n, m) - 1/9
end

println("Test 2")
n = 3
m = 5  # (1 + 1 + 3) x 3; (1 + 2 + 2) x 3, => 6 / 216 = 0.02777
for i = 1:3
    @show mc_dice(n, m) - 6/216
end


-------

**Bonus Question 1 (no marks)** Buffon's Needle Problem:

![buffon](http://mathworld.wolfram.com/images/eps-gif/BuffonNeedle_700.gif)

Suppose the $(x,y)$ plane is covered by vertical lines with distance $d = 1$ (see image). We randomly drop needles of length $l$ in the plane, i.e. at random positions and at random angles. What is the probability that a needle crosses the line?

Before you google for [a solution](http://mathworld.wolfram.com/BuffonsNeedleProblem.html), try to follow these instructions:

1. A randomly dropped needle can be modelled by two uniformly distributed random numbers $x \in [0, 1)$ for the $x$-coordinate of the needle centre and $\theta \in [0, \pi/2)$ for the angle at which it falls. Why is $x \in [0, 1)$ enough, why can we ignore $y$, and why is it sufficient to consider $\theta \in [0, \pi/2)$?  
2. Given the random numbers $x$, $\theta$, show that the end-points of the needle are given by 
$$
    \begin{pmatrix}
        x \pm \frac{l}{2}\cos\theta \\ 
        \pm \sin(\theta)/2
    \end{pmatrix}
$$

3. Write a function `needle(l)` that first generates two such random numbers and then, the end-point $x$-coordinate $x\pm = x \pm \frac{l}{2}\cos\theta$ and then returns `true` if the needle intersects the lines at $x = 0, 1$ and `false` if not.

4. Write a short script that uses the `needle` function that solves the Buffon Needle Problem to within an error of, say, $0.01$. You can compare your code against the analytic solution (which you should be able to find easily).

5. MORE CHALLENGING: You can try to extend the script to generate an image showing random needles with those that cross a line highlighted in a different color; see [this example](http://mathworld.wolfram.com/images/eps-gif/BuffonNeedleTosses_825.gif)

In [ ]:
# Soln 1-4
## BEGIN SOLUTION 
function needle(l)
    x = rand() 
    θ = rand() * π/2
    xp = x + l/2*cos(θ)
    xm = x - l/2*cos(θ)
    return ((xp >= 1) || (xm <= 0))
end 
l = 1
Nsamples = 1_000_000  
p = mean( needle(l) for n = 1:Nsamples )
@show p, 2/π
## END SOLUTION 


In [ ]:
# Soln 5.
## BEGIN SOLUTION 
function needle(l)
    x = rand() 
    y = rand() * 10 
    θ = π * (rand() - 0.5)
    xp = x + l/2*cos(θ)
    yp = y + l/2*sin(θ)
    xm = x - l/2*cos(θ)
    ym = y - l/2*sin(θ)
    crosses = ((xp >= 1) || (xm <= 0))
    x1 = rand(0:9)
    xp += x1 
    xm += x1
    return [xp, xm], [yp,ym], crosses
end 

l = 0.6
p = plot(xlim = (0, 10), ylim=(0, 10), label = "")
for x = 0:10 
    plot!([x,x], [0,10], lw=1, c=:black, label = "")
end
for n = 1:100
    x, y, crosses = needle(l)
    col = crosses ? (:red) : (:blue )
    plot!(x, y, lw=3, c = col, label = "")
end 
display(p)
## END SOLUTION

**Bonus Question 2 (no marks).** This question requires a little bit of mathematics background. Simply skip if it seem too technical for your taste.

Let $A \in \mathbb{R}^{N \times N}$ be a square and symmetric matrix ($A^T = A$), then a value $\lambda \in \mathbb{R}$ is an eigenvalue of $A$ if there exists a vector $v \in \mathbb{R}^N$ such that $A v = \lambda v$. Since $A$ is symmetric, there exist exactly $N$ eigenvalues (with multiplicities), $\lambda_1, \dots, \lambda_N$. 

Implement a short script that visualises the [Wigner Semi-circle Law](http://mathworld.wolfram.com/WignersSemicircleLaw.html): If $A_{ij} = A_{ji} \sim N(0, 1)$ are independent then the distribution of the eigenvalues has the probability density 
$$
  \lim_{N \to \infty} \mathbb{P}\big( \lambda_i/\sqrt{N} \in [a,b] \big) 
      = \frac{1}{2\pi} \int_a^b \sqrt{\big( 4 - x^2 \big)^{1/2}} dx.
$$
A possible solution could create a large symmetric random matrix (1000 x 1000 should be enough and much larger would be comutationally very expensive), compute its eigenvalues, create a histogram of their distribution and finally overlay the theoretical probability distribution function.

In [ ]:
## BEGIN SOLUTION 
N = 1_000
A = randn(N, N)
A = sqrt(0.5)*(A + A')
λ = real.(eigvals(A)) / sqrt(N)
histogram(λ, nbins = 50, normed = true, label="empirical")
σ = 1 # the variance of the normal distribution  
t = range(-2*σ, stop=2*σ, length=200)
plot!(t, sqrt.(4*σ^2 .- t.^2) / (2*π*σ^2), lw=3, label="semi-circle law")
## END SOLUTION

### Part b - Data Analysis

Start by loading the Titanic training dataset used in lectures, and applying the same transformations discussed to fill in missing values, as provided in the cell below:

In [ ]:
titanic = CSV.read("titanic.csv")

# Use an enumerated type for Survived column
@enum SurvivedType Dead=0 Survived=1
titanic[!,:Survived] = SurvivedType.(titanic[!,:Survived]);

# replaced missing Embarked entries with most common value
embarked=dropmissing(titanic,:Embarked)
embarked_mode = mode(embarked[!,:Embarked])
titanic[!,:Embarked]=coalesce.(titanic[!,:Embarked],embarked_mode)

# drop rows where Age is missing
titanic = dropmissing(titanic,:Age);

# add a Child feature where Age < 13
@enum ChildType Child=0 Adult=1
function classify_by_age(x)
  if x < 13
    return Child
  else
    return Adult
  end
end
titanic[!,:Child] = classify_by_age.(titanic[!,:Age]);

first(titanic,6)

**Question 5. (10%)**

The `Pclass` column gives the class passengers travelled in (1, 2 or 3), and can be viewed as a proxy for their socio-econonic class. Write a function `survival_by_class()` that takes the `titanic` dataset and the number of a class as arguments and returns the survival proportion in that class.

In [ ]:
function survival_by_class(titanic, class)
    mask = titanic[!,:Pclass] .== class
    return sum(titanic[mask,:Survived] .== Ref(Survived)) / sum(mask)
end

In [ ]:
println(@test isapprox(survival_by_class(titanic, 1), 0.656, atol=1e-2))
println(@test isapprox(survival_by_class(titanic, 2), 0.479, atol=1e-2))
println(@test isapprox(survival_by_class(titanic, 3), 0.239, atol=1e-2))

**Question 6a. (10%)**

Building on the example for the `Child` feature in the lecture, add a new column named `FamilySize` to the `titanic` dataset. The values should be the sum of the `SibSp` (Number of siblings/spouses aboard) and `Parch` (Number of parents/children aboard) plus one for the passenger themselves.

In [ ]:
titanic[!,:FamilySize] = titanic[!,:SibSp] + titanic[!,:Parch] .+ 1;

In [ ]:
println(@test mode(titanic[!,:FamilySize]) == 1)
println(@test isapprox(mean(titanic[!,:FamilySize]), 1.944, atol=1e-2))
println(@test isapprox(std(titanic[!,:FamilySize]), 1.484, atol=1e-2))

**Question 6b (5%).** 

Plot histograms to show the relationship between `FamilySize` and `Survived`. Write a short paragraph to describe what your results show.

In [ ]:
groupedbar(freqtable(titanic, :FamilySize, :Survived), label=[:Dead :Survived], 
            xlabel="Family size", ylabel="Frequency")

YOUR TEXT HERE

**Question 7a (10%)**

Add a new column named `Title` to the `titanic` dataset which gives the title of the passenger, e.g. `Mr.`, `Mrs.` etc, without the trailing `"."`. The French titles `Mlle` and `Mme` must be translated to `Miss` and `Mrs`, respectively, and `Ms` should be reassigned to `Miss`.

In [ ]:
## BEGIN SOLUTION
titanic[!,:Title] = replace.(titanic[!,:Name], (r"(.*, )|(\..*)"=>"",));

# Reassign Mlle, Ms, and Mme accordingly
for (from, to) in [("Mlle", "Miss"),
                   ("Ms", "Miss"),
                   ("Mme", "Mrs")]
    titanic[titanic[!,:Title] .== from, :Title] .= to
end
   
# Optionally, collapse titles with very low counts
for title in ["Dona", "Lady", "the Countess", "Capt", "Col", "Don", 
              "Dr", "Major", "Rev", "Sir", "Jonkheer"]
    titanic[titanic[!,:Title] .== title, :Title] .= "Rare Title"
end
countmap(titanic[!,:Title])
## END SOLUTION

In [ ]:
println(@test eltype(titanic[!,:Title]) == String)
println(@test sum(titanic[!,:Title] .== "Mr") == 398)
println(@test sum(titanic[!,:Title] .== "Miss") == 149)
println(@test sum(titanic[!,:Title] .== "Mrs") == 109)

**Question 7b (5%)**

Which title is associated with the lowest chance of survival? Plot a graph to evidence your answer.

"Mr." is the most dangerous title, as people with that title have the lowest chance of survival.

In [ ]:
groupedbar(freqtable(titanic, :Title, :Survived), label=[:Dead :Survived], 
           xticks=(1:5, sort(levels(titanic[!,:Title]))))

**Question 8. (10%)**

Fit a normal distribution to the `Age` variable from the `titanic` dataset.  Compute its mean and standard deviation and store them in the variables `mean_age` and `std_age`.

*Optional extension:* plot a comparison of the data and fit and think about how these results could be used to avoid deleting the rows where the age is missing.

In [ ]:
fit_norm = fit(Normal, titanic[!,:Age])
@show mean_age = mean(fit_norm)
@show std_age = std(fit_norm)
density(titanic[!,:Age], lw=2, label="Age data")
plot!(fit_norm, lw=2, label="Normal fit")

In [ ]:
## BEGIN HIDDEN TEST 
println(@test mean_age ≈ 29.69911764705882)
println(@test std_age ≈ 14.516321150817317)
## END HIDDEN TEST

**Bonus Question (no marks)**

Apply the `DecisionTree` from the lecture to build a classifier for surival on the Titanic, using all the features constructed so far as input, and reserving one third of the training data for testing. Store your results in the variable `predictions`. How accurate is your predictive model, and which features do you think are the most important?

In [ ]:
Random.seed!(1) # seed the RNG for reproducible results
train = rand(nrow(titanic)) .< 2/3
test = .!train 
@show sum(train), sum(test)

# use Survived column for the training data - feature selection is up to you!
labels = convert(Array, titanic[:, :Survived])

In [ ]:
features = convert(Matrix, titanic[:, [:Age, :Sex, :Child, :FamilySize]])
model = build_tree(labels[train], features[train,:])
predictions = apply_tree(model, features[test,:]);

In [ ]:
_toint(s::SurvivedType) = (s == Dead ? 0 : 1)
cm = confusion_matrix(_toint.(labels[test]), _toint.(predictions))
@show cm
@test cm.accuracy > 0.70 # can you beat 70% accuracy?